# Gradient Descent with Autograd and Backpropagation

In [1]:
import numpy as np

# Manual GD

In [3]:

# Linear regression
# f = w * x 

# here : f = 2 * x
X = np.array([1, 2, 3, 4], dtype=np.float32)
Y = np.array([2, 4, 6, 8], dtype=np.float32)

w = 0.0

# Model output
def forward(x):
    return w * x

# Loss = MSE
def loss(y, y_pred):
    return ((y_pred - y)**2).mean()

# Gradient
# MSE = 1/N * (w*x - y)**2
# dJ/dw = 1/N * 2 * (w*x - y) * x

def gradient(x, y, y_predicted):
    return (2*np.dot(x, y_predicted-y)).mean()

print(f"Prediction before training: f(5) = {forward(5):.3f}")
# Training 
learning_rate = 0.01
n_iters = 20

for epoch in range(n_iters):
    y_pred = forward(X)

    l = loss(Y, y_pred)

    #gradients
    dw = gradient(X,Y, y_pred)

    #update weights:
    w-=learning_rate*dw

    if epoch % 2 == 0:
        print(f'epoch {epoch + 1}: w = {w:.3f}, loss={l:.8f}')

print(f"Prediction after training: f(5) = {forward(5):.3f}")

Prediction before training: f(5) = 0.000
epoch 1: w = 1.200, loss=30.00000000
epoch 3: w = 1.872, loss=0.76800019
epoch 5: w = 1.980, loss=0.01966083
epoch 7: w = 1.997, loss=0.00050332
epoch 9: w = 1.999, loss=0.00001288
epoch 11: w = 2.000, loss=0.00000033
epoch 13: w = 2.000, loss=0.00000001
epoch 15: w = 2.000, loss=0.00000000
epoch 17: w = 2.000, loss=0.00000000
epoch 19: w = 2.000, loss=0.00000000
Prediction after training: f(5) = 10.000


## GD with Torch AutoGrad

In [4]:
import torch

# Linear regression
# f = w * x 

# here : f = 2 * x
X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

# model output
def forward(x):
    return w * x

# loss = MSE
def loss(y, y_pred):
    return ((y_pred - y)**2).mean()

print(f'Prediction before training: f(5) = {forward(5).item():.3f}')

# Training
learning_rate = 0.01
n_iters = 100

for epoch in range(n_iters):
    # predict = forward pass
    y_pred = forward(X)

    # loss
    l = loss(Y, y_pred)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    #w.data = w.data - learning_rate * w.grad
    with torch.no_grad(): #since we dont want weight updating stay within computational graph
        w -= learning_rate * w.grad
    
    # zero the gradients after updating
    w.grad.zero_()

    if epoch % 10 == 0:
        print(f'epoch {epoch+1}: w = {w.item():.3f}, loss = {l.item():.8f}')

print(f'Prediction after training: f(5) = {forward(5).item():.3f}')


Prediction before training: f(5) = 0.000
epoch 1: w = 0.300, loss = 30.00000000
epoch 11: w = 1.665, loss = 1.16278565
epoch 21: w = 1.934, loss = 0.04506890
epoch 31: w = 1.987, loss = 0.00174685
epoch 41: w = 1.997, loss = 0.00006770
epoch 51: w = 1.999, loss = 0.00000262
epoch 61: w = 2.000, loss = 0.00000010
epoch 71: w = 2.000, loss = 0.00000000
epoch 81: w = 2.000, loss = 0.00000000
epoch 91: w = 2.000, loss = 0.00000000
Prediction after training: f(5) = 10.000


## Training Pipeline

In [9]:
import torch
import torch.nn as nn
# Linear regression
# f = w * x 

# here : f = 2 * x
#Input:  ROW = Number of Training instances; COL = Number of Features

X = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32) 
Y = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

X_test = torch.tensor([5], dtype=torch.float32)

n_samples, n_features = X.shape
print(n_samples, n_features)


input_size = n_features
output_size = n_features


#Customize the model
class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        #define layers
        self.lin = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.lin(x)

model = LinearRegression(input_size, output_size)

print(f'Prediction before training: f(5) = {model(X_test).item():.3f}')

# Training
learning_rate = 0.01
n_iters = 100

#Define Loss Function & optimizer
loss = nn.MSELoss()

#optimizer = torch.optim.SGD([w], lr=learning_rate)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(n_iters):
    # predict = forward pass
    y_pred = forward(X)

    # loss
    l = loss(Y, y_pred)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    # w.data = w.data - learning_rate * w.grad
    # with torch.no_grad(): #since we dont want weight updating stay within computational graph
    #     w -= learning_rate * w.grad
    optimizer.step() # optimizer will take care the above code to update weights

    # zero the gradients after updating
    #w.grad.zero_()
    optimizer.zero_grad()

    if epoch % 10 == 0:
        [w, b] = model.parameters() #unpack model parameters
        print(f'epoch {epoch+1}: w = {w[0][0].item():.3f}, loss = {l.item():.8f}')

print(f'Prediction after training: f(5) = {model(X_test).item():.3f}')

4 1
Prediction before training: f(5) = 1.800
epoch 1: w = 0.427, loss = 0.00000000
epoch 11: w = 1.690, loss = 0.99531841
epoch 21: w = 1.939, loss = 0.03857818
epoch 31: w = 1.988, loss = 0.00149527
epoch 41: w = 1.998, loss = 0.00005796
epoch 51: w = 2.000, loss = 0.00000225
epoch 61: w = 2.000, loss = 0.00000009
epoch 71: w = 2.000, loss = 0.00000000
epoch 81: w = 2.000, loss = 0.00000000
epoch 91: w = 2.000, loss = 0.00000000
Prediction after training: f(5) = 9.664
